# Functionality: visualization of functional connectivity

## Import Packages 

In [1]:
import os, sys
import numpy as np
import pandas as pd

import pickle

import matplotlib.pyplot as plt

from igraph.drawing.text import TextDrawer
import cairo

In [2]:
# import_nbmodule used for import package in .ipynb
import import_nbmodule

# extract the exp folder path
currfolder = os.getcwd()
codefolder = currfolder[0 : currfolder.find('code')+len('code')]

# add path the exp folder
sys.path.append(codefolder)


# import animal_folder function in util/folder_extract.pynb 
from util.folder_extract import exp_subfolders, code_corresfolder

# import graphVisual in connAnalyTool/graphVisual.pynb 
from connAnalyTool import graphVisual

importing Jupyter notebook from /Users/linglingyang/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/util/folder_extract.ipynb
importing Jupyter notebook from /Users/linglingyang/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/connAnalyTool/graphVisual.ipynb


## Preparasion

### exp subfolders & code_corresfolder

In [3]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [4]:
nb_name = nb_name[0: nb_name.find('.ipynb')]

# corresfolder
corresfolder,correparentfolder = code_corresfolder(os.getcwd(), nb_name)

### global parameters

In [5]:
# frequency
freq = [19, 21]
strfreq = 'freq' + str(freq[0]) + '_' + str(freq[1])


# eventphase
eventphase = 'baseline'

filesUsed = 'alldays' # 'samedays', or 'closedays' or 'alldays' or 'remaindays'

In [6]:
### text setup for brain areas ###
texts_area = dict()

texts_area['VPLo'] = [20, 40, 20]

texts_area['VLo'] = [20, 120, 20]

texts_area['VA'] = [20, 200, 20]

texts_area['Sensory Motor'] = [50, 350, 20]

texts_area['SMA'] = [20, 500, 20]

texts_area['M1'] = [150, 590, 20]

texts_area['PMC'] = [310, 590, 20]

texts_area['STN'] = [280, 140, 20]

texts_area['GP'] = [360, 250, 20]

In [7]:
WithinM1NoShow = False
STNConnectOnly = False
GPConnectOnly = False
CorticalConnectOnly = False

### input setup

In [8]:
inputfolder = os.path.join(correparentfolder, 'm1_SKT_calcciCOH'  + 
                           '_Fre' + str(freq[0]) + '-' + str(freq[1]) +'Hz' + '_' + eventphase )
filename = 'ciCOH_STK_'  + 'freq_' + str(freq[0]) + '_' + str(freq[1]) + '_' + eventphase


if filesUsed == 'samedays':
    filename = filename + '_samedays'
elif filesUsed == 'closedays':
    filename = filename + '_closedays'
elif filesUsed == 'remaindays':
    filename = filename + '_remaindays'

file = os.path.join(inputfolder, filename + '.pickle')

### save setup

In [9]:
savefolder = corresfolder
savefile_prefix_origin = 'ciCOH_' + eventphase + '_' + strfreq 

if filesUsed == 'samedays':
    savefile_prefix_origin = savefile_prefix_origin + '_samedays'
elif filesUsed == 'closedays':
    savefile_prefix_origin = savefile_prefix_origin + '_closedays'
elif filesUsed == 'remaindays':
    savefile_prefix_origin = savefile_prefix_origin + '_remaindays'

## Load ciCOH and Channel

In [10]:
fp = open(file, 'rb')
ciCOH = pickle.load(fp)

In [11]:
# ciCOH
ciCOH_normal, ciCOH_mild = ciCOH['normal'], ciCOH['mild']

# channel information
chnInf_normal, chnInf_mild = ciCOH['chnInf_normal'], ciCOH['chnInf_mild']

## Show Selection 

## ciCOH Visual

In [12]:
def ciCOH_visual_save(ciCOH, chn_area, chn_coord, str_recchnishow, lowweight, tobesavedfile, texts, 
                      edgeWider = False, edgeWiderMul = 2):
    """
        
        Args:
            
            ciCOH (np.sdarray): ciCOH matrix (nchns, nchns)
            
            chn_area (list): a list of string representing brain area for each channel, len(chn_area) = nchns
            
            chn_coord (numpy.ndarray): x, y coordinates for each channel, shape = (nchns, 2)
            
            chn_recchni(list): showed recording chni for each channel, shape = (nchns, )
            
            lowweight: the threshold lowweight, only weight>lowweight is treated as connection
            
            tobesavedfile: file to save the visualized figure
            
            texts:
            
            edgeWider:
            
            edgeWiderMul: if edgeWider = True, this value is used for wider the edge width
            
        Output:
            the visualizaton of ciCOH is saved in tobesavedfile
            
    """
    
    weight = abs(ciCOH)

    # weight > lowweight
    weight[np.where(weight<lowweight)] =0
    
    graph = graphVisual.graph_create(weight, vsarea = chn_area, vscoord = chn_coord, vschni = str_recchnishow)

    # set graph visualization style
    visual_style = dict()
    if edgeWider:
        visual_style['edge_width'] = [w * edgeWiderMul for w in graph.es['weight']]
    
    visual_style = graphVisual.graph_style(graph, visual_style)
    

    igplot = graphVisual.graph_plot(graph, visual_style, texts = texts)
    
    igplot.save(tobesavedfile)
    
    return igplot

## Save

In [13]:
# threshold lowweight
lowweight = 0.05

edgeWider = True
edgeWiderMul = 2

In [14]:
savefile_prefix = savefile_prefix_origin

if STNConnectOnly:
    ciCOH_normal, ciCOH_mild = ciCOH_normal_onlySTN, ciCOH_mild_onlySTN
    savefile_prefix = savefile_prefix_origin + '_onlySTN'

if GPConnectOnly:
    ciCOH_normal, ciCOH_mild = ciCOH_normal_onlyGP, ciCOH_mild_onlyGP
    savefile_prefix = savefile_prefix_origin + '_onlyGP'
    
if CorticalConnectOnly:
    ciCOH_normal, ciCOH_mild = ciCOH_normal_onlyCortical, ciCOH_mild_onlyCortical
    savefile_prefix = savefile_prefix_origin + '_onlyCortical'

### normal

In [15]:
"""
    normal condition
"""

chnInf_recchni, chnInf_matchni = chnInf_normal['chnInf_recchni'], chnInf_normal['chnInf_matchni']
chnInf_brainarea, chnInf_coord = chnInf_normal['chnInf_brainarea'], chnInf_normal['chnInf_coord']


# save file config
savefilename = savefile_prefix+ '_lowweight' + str(lowweight) + '_normal' + '.png'
savefile = os.path.join(savefolder, savefilename)

# printed text config
texts = texts_area.copy()


# visualize and save
igplot = ciCOH_visual_save(ciCOH_normal, 
                           chn_area = chnInf_brainarea, 
                           chn_coord = chnInf_coord,
                           str_recchnishow = chnInf_recchni,
                           lowweight = lowweight,  
                           tobesavedfile = savefile, 
                           texts = texts,
                           edgeWider = edgeWider, edgeWiderMul = edgeWiderMul)

igplot.show()

print("Figure saved to " + savefile)


# del 
del savefilename, savefile, texts, 
del chnInf_recchni, chnInf_matchni,  chnInf_brainarea, chnInf_coord

Figure saved to /Users/linglingyang/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/pipeline/NHP_Bug/1_dataAnaly/FCAnaly/SKT/m2_SKT_visualFC_Fre19-21Hz_baseline/ciCOH_baseline_freq19_21_lowweight0.05_normal.png


### mild

In [16]:
#### delete the mild channels those not in normal 
ciCOH_mild_orign = ciCOH_mild
idx_MildChnsnotinNormal = [13, 18, 19, 21, 22, 25, 28]

ciCOH = ciCOH_mild
ciCOH = np.delete(ciCOH, idx_MildChnsnotinNormal, axis = 0)
ciCOH = np.delete(ciCOH, idx_MildChnsnotinNormal, axis = 1)
ciCOH_mild = ciCOH
del ciCOH

In [17]:
"""
    mild condition
"""

condition = 'mild'

chnInf_recchni, chnInf_matchni = chnInf_normal['chnInf_recchni'], chnInf_normal['chnInf_matchni']
chnInf_brainarea, chnInf_coord = chnInf_normal['chnInf_brainarea'], chnInf_normal['chnInf_coord']


# save file config
savefilename = savefile_prefix + '_lowweight' + str(lowweight) + '_' + condition + '.png'
savefile = os.path.join(savefolder, savefilename)

# printed text config
texts = texts_area.copy()

# visualize and save
igplot = ciCOH_visual_save(ciCOH_mild, 
                           chn_area = chnInf_brainarea, 
                           chn_coord = chnInf_coord,
                           str_recchnishow = chnInf_recchni,
                           lowweight = lowweight,  
                           tobesavedfile = savefile, 
                           texts = texts,
                           edgeWider = edgeWider, edgeWiderMul = edgeWiderMul)

igplot.show()

print("Figure saved to " + savefile)

# del 
del savefilename, savefile, texts

Figure saved to /Users/linglingyang/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/pipeline/NHP_Bug/1_dataAnaly/FCAnaly/SKT/m2_SKT_visualFC_Fre19-21Hz_baseline/ciCOH_baseline_freq19_21_lowweight0.05_mild.png
